# WhisperTS JSON vers SRT

In [1]:
import re,codecs,json,glob
import pandas as pd

In [2]:
repJson="/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/JSON/"
fTranscript=glob.glob(repJson+"*.json")[0]
pause=0.200
with open(fTranscript) as inFile:
    whisper=json.load(inFile)

IndexError: list index out of range

In [ ]:
def second2timestamp(seconds):
    sHours=int(seconds/3600)
    sMinutes=int((seconds-sHours*3600)/60)
    sSeconds=int((seconds-sHours*3600-sMinutes*60))
    sMilliseconds=int((seconds%1)*1000)
    return "%02d:%02d:%02d,%03d"%(sHours,sMinutes,sSeconds,sMilliseconds)

In [ ]:
def timestamp2second(timestamp):
    m=re.match("(\d\d):(\d\d):(\d\d),(\d\d\d)",timestamp)
    if m:
        seconds=(int(m.group(1))*60+int(m.group(2)))*60+int(m.group(3))+float(m.group(4))/1000
        return seconds
    else:
        print("problème de timestamp")
        return

In [ ]:
def getWhisper(elements):
    lElements=[]
    if elements=="segments":
        for element in whisper[elements]:
            lElements.append((second2timestamp(element["start"]),
                              second2timestamp(element["end"]),
                              element["text"],
                              element["confidence"]))
    elif elements=="words":
        for segment in whisper["segments"]:
            for element in segment[elements]:
                lElements.append((second2timestamp(element["start"]),
                                  second2timestamp(element["end"]),
                                  element["text"],
                                  element["confidence"]))        
    return lElements

In [ ]:
tMots=getWhisper("words")
dfMots=pd.DataFrame(tMots,columns="start end mot confidence".split(" "))
dfMots

In [ ]:
dfSegments=pd.DataFrame(getWhisper("segments"),columns="start end mot confidence".split(" "))
dfSegments

In [ ]:
second2timestamp(3665.547)

In [ ]:
srtLines=[]
for i,(start,end,mot,confidence) in enumerate(tMots):
    n=i+1
    srtLines.append(str(n))
    srtLines.append(start+" --> "+end)
    srtLines.append(mot)
    srtLines.append("")


In [ ]:
with codecs.open(fTranscript.replace(".json","-mots.srt"),"w",encoding="utf8") as outFile:
    for srtLine in srtLines:
        outFile.write(srtLine+"\n")

In [ ]:
srtPhrases=[]
srtMots=[]
bPhrase=True
pStart=0
pEnd=0
for i,(start,end,mot,confidence) in enumerate(tMots):
    
    if bPhrase:
        print ("start",start)
        pStart=start
        srtMots=[]
        bPhrase=False
    m=re.search(r"[.?!]$",mot)
    if m:
        print("end",end)
        pEnd=end
        srtMots.append(mot)
        pPhrase=" ".join(srtMots)
        srtPhrases.append((pStart,pEnd,pPhrase))
        bPhrase=True
    elif not bPhrase:
        srtMots.append(mot)

In [ ]:
srtPhrases


In [ ]:
srtPhrasesLines=[]
for i,(start,end,phrase) in enumerate(srtPhrases):
    n=i+1
    srtPhrasesLines.append(str(n))
    srtPhrasesLines.append(start+" --> "+end)
    srtPhrasesLines.append(phrase)
    srtPhrasesLines.append("")
    
with codecs.open(fTranscript.replace(".json","-phrases.srt"),"w",encoding="utf8") as outFile:
    for srtPhrasesLine in srtPhrasesLines:
        outFile.write(srtPhrasesLine+"\n")

In [ ]:
srtTours=[]
srtMots=[]
bTour=True
prevEnd=0
pStart=0
pEnd=0
for i,(start,end,mot,confidence) in enumerate(tMots):
    sStart=timestamp2second(start)
    sEnd=timestamp2second(end)
    if sStart-prevEnd>pause and prevEnd!=0:
        pEnd=prevEnd
        pTour=" ".join(srtMots)
        srtTours.append((pStart,pEnd,pTour))
        pStart=sStart
        prevEnd=sEnd
        srtMots=[mot]
        bPhrase=True
    else:
        srtMots.append(mot)
        prevEnd=sEnd
pTour=" ".join(srtMots)
srtTours.append((pStart,pEnd,pTour))

In [ ]:
srtToursLines=[]
for i,(start,end,phrase) in enumerate(srtTours):
    n=i+1
    srtToursLines.append(str(n))
    srtToursLines.append(second2timestamp(start)+" --> "+second2timestamp(end))
    srtToursLines.append(phrase)
    srtToursLines.append("")
    
with codecs.open(fTranscript.replace(".json","-tours.srt"),"w",encoding="utf8") as outFile:
    for srtToursLine in srtToursLines:
        outFile.write(srtToursLine+"\n")